In [27]:
import pandas as pd
import requests
from datetime import datetime, date, timedelta
from pytz import timezone
from IPython.display import JSON
from api_keys import rapidApi_key as fkey
from api_keys import mysql_key as sql_key

In [11]:
# call airports_df to use in this file
airports_df = pd.read_csv('airports.csv')
print(airports_df.info())
airports_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   icao     6 non-null      object
 1   city_id  6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 228.0+ bytes
None


,icao,city_id
0,EDDB,1
1,EGLC,2
2,EGKK,2
3,EGSS,2
4,LTFM,4
5,LTFJ,4


In [12]:
# get today and tomorrow's date since we wants flight info for the next day
today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
tomorrow = (today + timedelta(days=1))

In [13]:
times = [["00:00","11:59"],["12:00","23:59"]]

In [14]:
icao_list = airports_df['icao'].to_list()


In [15]:
icao_list

['EDDB', 'EGLC', 'EGKK', 'EGSS', 'LTFM', 'LTFJ']

In [16]:
element_to_exclude = 'EGLC'



In [17]:
updated_icao_list = [icao for icao in icao_list if icao != element_to_exclude]
updated_icao_list

['EDDB', 'EGKK', 'EGSS', 'LTFM', 'LTFJ']

In [18]:
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}
headers = {
	"X-RapidAPI-Key": fkey,
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [19]:
flights_dict = {'icao':[],
                'departure_airport':[],
                'local_arrival_time':[],
                'arrival_terminal': [],
                'airline': [],
                'flight_number': [],
                'flight_status': [],    
                }

for time in times:

    for icao in updated_icao_list:
        flight_url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", flight_url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()

        for flight in flight_json['arrivals']:
            flights_dict['icao'].append(icao)
            flights_dict['departure_airport'].append(flight['departure']['airport']['name'])
           
            flights_dict['airline'].append(flight['airline']['name'])
            flights_dict['flight_number'].append(flight['number'])
            flights_dict['flight_status'].append(flight['status']) 
            
            try:
                    flights_dict['arrival_terminal'].append(flight['arrival']['terminal'])
            except:
                    flights_dict['arrival_terminal'].append('unknown')
            try:
                     flights_dict['local_arrival_time'].append(flight['departure']['scheduledTime']['local'])
            except:
                    flights_dict['local_arrival_time'].append(pd.NaT)
                                  
              

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [20]:
response.json()

{'arrivals': [{'departure': {'airport': {'icao': 'ORBI',
     'iata': 'BGW',
     'name': 'Baghdad'},
    'scheduledTime': {'utc': '2023-10-16 05:50Z',
     'local': '2023-10-16 08:50+03:00'},
    'quality': ['Basic']},
   'arrival': {'scheduledTime': {'utc': '2023-10-16 09:00Z',
     'local': '2023-10-16 12:00+03:00'},
    'quality': ['Basic']},
   'number': 'IF 201',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Boeing 737-900'},
   'airline': {'name': 'Fly Baghdad', 'iata': 'IF', 'icao': 'FBA'}},
  {'departure': {'airport': {'icao': 'LTAF', 'iata': 'ADA', 'name': 'Adana'},
    'scheduledTime': {'utc': '2023-10-16 07:30Z',
     'local': '2023-10-16 10:30+03:00'},
    'quality': ['Basic']},
   'arrival': {'scheduledTime': {'utc': '2023-10-16 09:00Z',
     'local': '2023-10-16 12:00+03:00'},
    'quality': ['Basic']},
   'number': 'TK 7261',
   'status': 'Unknown',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircra

In [57]:
#flight_details = response.json()
#flight_details['arrivals'][0]['arrival']['terminal']#['name']

In [21]:
flights_df = pd.DataFrame(flights_dict)

In [22]:
flights_df

,icao,departure_airport,local_arrival_time,arrival_terminal,airline,flight_number,flight_status
0,EDDB,İzmir,2023-10-16 03:40+03:00,1,Sun Express,XQ 966,Expected
1,EDDB,Beijing,2023-10-16 02:40+08:00,1,Hainan,HU 489,Expected
2,EDDB,Riga,2023-10-16 07:05+03:00,1,airBaltic,BT 211,Expected
3,EDDB,New York,2023-10-15 17:00-04:00,1,Delta Air Lines,DL 92,Expected
4,EDDB,Cologne,2023-10-16 06:20+02:00,1,Eurowings,EW 12,Expected
...,...,...,...,...,...,...,...
1941,LTFJ,Gaziantep,2023-10-16 21:55+03:00,unknown,AnadoluJet,TK 7273,Unknown
1942,LTFJ,Şanlıurfa,2023-10-16 21:50+03:00,unknown,Pegasus,PC 2421,Unknown
1943,LTFJ,Nicosia,2023-10-16 22:20+03:00,unknown,Pegasus,PC 1925,Unknown
1944,LTFJ,Adana,2023-10-16 22:20+03:00,unknown,AnadoluJet,TK 7263,Unknown


In [23]:
#export to csv
flights_df.to_csv('flights.csv', index=False)

In [24]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   icao                1946 non-null   object
 1   departure_airport   1946 non-null   object
 2   local_arrival_time  1823 non-null   object
 3   arrival_terminal    1946 non-null   object
 4   airline             1946 non-null   object
 5   flight_number       1946 non-null   object
 6   flight_status       1946 non-null   object
dtypes: object(7)
memory usage: 106.6+ KB


## Connect to the MySQL database

In [25]:
flights_df = pd.read_csv('flights.csv')
flights_df 

,icao,departure_airport,local_arrival_time,arrival_terminal,airline,flight_number,flight_status
0,EDDB,İzmir,2023-10-16 03:40+03:00,1,Sun Express,XQ 966,Expected
1,EDDB,Beijing,2023-10-16 02:40+08:00,1,Hainan,HU 489,Expected
2,EDDB,Riga,2023-10-16 07:05+03:00,1,airBaltic,BT 211,Expected
3,EDDB,New York,2023-10-15 17:00-04:00,1,Delta Air Lines,DL 92,Expected
4,EDDB,Cologne,2023-10-16 06:20+02:00,1,Eurowings,EW 12,Expected
...,...,...,...,...,...,...,...
1941,LTFJ,Gaziantep,2023-10-16 21:55+03:00,unknown,AnadoluJet,TK 7273,Unknown
1942,LTFJ,Şanlıurfa,2023-10-16 21:50+03:00,unknown,Pegasus,PC 2421,Unknown
1943,LTFJ,Nicosia,2023-10-16 22:20+03:00,unknown,Pegasus,PC 1925,Unknown
1944,LTFJ,Adana,2023-10-16 22:20+03:00,unknown,AnadoluJet,TK 7263,Unknown


In [28]:
host = '127.0.0.1'
schema = 'gans_local' # The name of your database (schema) might be different
user = 'root'
password = sql_key # YOUR password
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [29]:
flights_df
print(flights_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1946 entries, 0 to 1945
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   icao                1946 non-null   object
 1   departure_airport   1946 non-null   object
 2   local_arrival_time  1823 non-null   object
 3   arrival_terminal    1946 non-null   object
 4   airline             1946 non-null   object
 5   flight_number       1946 non-null   object
 6   flight_status       1946 non-null   object
dtypes: object(7)
memory usage: 106.6+ KB
None


In [30]:
flights_df['local_arrival_time'] = pd.to_datetime(flights_df['local_arrival_time'],utc=True)

In [31]:
flights_df.to_sql('flights',con=con,if_exists='append',index=False)

1946

In [25]:
flights_df
print(flights_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1822 entries, 0 to 1821
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   icao                1822 non-null   object             
 1   departure_airport   1822 non-null   object             
 2   local_arrival_time  1729 non-null   datetime64[ns, UTC]
 3   arrival_terminal    1822 non-null   object             
 4   airline             1822 non-null   object             
 5   flight_number       1822 non-null   object             
 6   flight_status       1822 non-null   object             
dtypes: datetime64[ns, UTC](1), object(6)
memory usage: 99.8+ KB
None
